# This Notebook scrapes and cleans the Toronto Neighborhoods data from Wikipedia

## 1-  First the required libraries are imported

In [10]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup as bs

## 2-  This step opens the source html and scrape the data

In [11]:
# Open the HTML File from Wikipedia
htmlfile = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

# Reads the tables that contain the word 'Borough'
dfs = pd.read_html(htmlfile, match='Borough')

# Assign the first data frame extracted/scraped to a the data frame 'tndf' and displays the first 5 rows.
tndf = dfs[0]

tndf.head()

#The resulting dataframe 'tndf' has the requested format:

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 3-  This step cleans the data for 'Not assigned' Boroughs

In [12]:
# Find rows where the Borough is not assigned

indexNames = tndf[tndf['Borough'] == 'Not assigned'].index

# Drop rows where the Borough is not assigned

tndf.drop(indexNames, inplace=True)

# Reset index of the Data Frame

tndf.reset_index(drop=True, inplace=True)

# Displays the data frame first 5 rows

tndf.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 4- The below code looks for neighborhoods  that have the same postal code area (postal code area in more than one row) and combine them into one row. 
There are no such cases anymore on the Wikipedia page, but the code below would take care of new cases if they arise. 

In [13]:
# Find Postal Codes that are shown more than once

indexpc = tndf['Postal Code'].value_counts() > 1 #I dentify the Postal Codes that are more than one time on the data frame tndf 

pc = indexpc[indexpc] 

pci = pc.index # Holds a list of the Postal Codes that are more than once on the data frame tndf

# For each Postal Code that is shown more than once get the rows index

for i in pci:
   

    udf = tndf[tndf['Postal Code']==i]

    indexdf = udf.index
    
    countindex = len(udf.index)
    

    # Reset the variable for concatenating the Neighborhood names and the counter for updating the data frame rows     
    anb = ""
    count = 1

    for j in indexdf:
        
       
        nb = tndf['Neighborhood'].loc[j]
        if count == 1:
            anb = nb
        else:    
            anb = nb+", "+anb  # Concatenates neighborhoods that share the same Postal Code Area
        if count < countindex:
            tndf.drop(j, inplace=True) # Deletes rows for neighborhoods that share the same Postal Code Area unless is the last row
        elif count == countindex:
            tndf['Neighborhood'].loc[j] = anb # Updates the Neiborhood with the comma separated list of neighborhoods that share the same postal code area
        
        count = count + 1


# Reset index of the Data Frame

tndf.reset_index(drop=True, inplace=True)

tndf.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 5- Next step is to Clean the data for 'Not assigned' Neighborhoods assigning the Borough Name.
No cases are currently on the data, but in case there are the following code will fix it.

In [14]:
# Find rows where the Neighborhood is 'Not assigned'

indexNames2 = tndf[tndf['Neighborhood'] == 'Not assigned'].index

# Assign the Borough name to the Neighborhood where the Neighborhood is not assigned

for i in indexNames2:

    b = tndf['Borough'].loc[i]
    tndf['Neighborhood'].loc[i] = b  


## 6- Using .shape method to print the number of rows of the dataframe

In [15]:
tndf.shape

(103, 3)

## 7- Get the geographical coordinates via the CSV file

In [16]:
# Defines the CSV
geocsvhtml ='http://cocl.us/Geospatial_data'

geoco = pd.read_csv(geocsvhtml)


In [6]:
geoco

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


## 8- Create a data frame with the Toronto Neighborhoods and geographical coordinates


In [20]:
df = pd.merge(tndf, geoco, on=['Postal Code'], how='left')

df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
